### Đã xử lý được trường hợp ko bắt đầu bằng từ khóa

In [25]:
from docx import Document
import re

def contains_exact_keyword(text, keyword):
    # Sử dụng regex để tìm từ khóa đứng riêng biệt và chính xác về chữ hoa/thường
    pattern = r'\b' + re.escape(keyword) + r'\b'
    return bool(re.search(pattern, text))

def remove_comments(text):
    # Loại bỏ phần comment bắt đầu bằng '--' hoặc '---'
    return re.split(r'--|//', text)[0].strip()
    
def remove_text_before_keyword(text, keyword):
    # Sử dụng regex để tách chuỗi tại từ khóa và chỉ giữ lại phần sau từ khóa
    parts = re.split(r'(?i)\b' + re.escape(keyword) + r'\b', text)
    if len(parts) > 1:
        return keyword + parts[-1].strip()  # Thêm lại từ khóa vào phần sau
    return text  # Nếu từ khóa không có trong text thì trả về text gốc

def format_sql(text):
    # Thêm dấu cách giữa từ khóa SQL và phần tiếp theo nếu cần
    keywords_with_space = ['WITH', 'SELECT', 'UPDATE', 'CASE WHEN', 'CASE', 'WHERE', 'UNION ALL', 'JOIN']
    for keyword in keywords_with_space:
        text = re.sub(rf'\b{keyword}\s*', f'{keyword} ', text, flags=re.IGNORECASE)
    return text

def extract_sql_code(docx_file):
    doc = Document(docx_file)
    sql_code_blocks = []
    current_block = []
    
    # Danh sách các từ khóa SQL để nhận diện bắt đầu đoạn SQL
    sql_keywords = ('WITH', 'SELECT', 'UPDATE', 'CASE WHEN', 'CASE', 'WHERE', 'UNION ALL', 'JOIN')
    sql_dml_keywords = ('SELECT', 'UPDATE', 'INSERT', 'DELETE')
    
    paragraphs = doc.paragraphs

    for i, para in enumerate(paragraphs):
        text = para.text.strip()
        text = remove_comments(text)  # Loại bỏ phần comment nếu có
        
        # Nếu đoạn văn trống, kết thúc khối SQL hiện tại nếu cần
        if not text or para.style.name.startswith('Heading'):
            # Kiểm tra nếu đoạn trước là từ khóa và đoạn tiếp theo cũng trống thì không kết thúc
            if current_block and not any(kw in current_block[-1].upper() for kw in sql_keywords):
                sql_code_blocks.append("\n".join(current_block))
                current_block = []
            continue
        
        # Kiểm tra nếu đoạn văn bắt đầu bằng từ khóa SQL in hoa hoàn toàn
        # if any(text.startswith(kw) for kw in sql_keywords) and text == text.upper():
        #     current_block.append(text)
        
        if any(contains_exact_keyword(text, kw) for kw in sql_dml_keywords):
            # Nếu text chứa từ khóa SQL, xóa phần đứng trước từ khóa
            for keyword in sql_dml_keywords:
                if contains_exact_keyword(text, keyword):
                    text = remove_text_before_keyword(text, keyword)
                    text = format_sql(text)
                    break
            current_block.append(text)
        
        # Tiếp tục thêm vào block hiện tại nếu đang trong khối SQL
        elif current_block:
            current_block.append(text)

        # Nếu đoạn văn kết thúc bằng dấu chấm phẩy, thì xem như khối SQL đã hoàn tất
        if text.endswith(';'):
        #if ';' in text:
            sql_code_blocks.append("\n".join(current_block))
            current_block = []
            continue
                
    # Trường hợp cuối cùng còn block chưa kết thúc
    if current_block:
        sql_code_blocks.append("\n".join(current_block))

    return sql_code_blocks

# Đọc file Word và trích xuất SQL code
sql_codes = extract_sql_code('JOB nghiệp vụ 2.11.docx')
for i, sql in enumerate(sql_codes):
    print(f"Query {i}:\n")
    print(sql)
    print("\n" + "="*80 + "\n")


Query 0:

SELECT dữ liệu từ bảng TBL_SLIP_MARK bảng group by theo INPUTTER, DEPARTMENT_CODE, COMPANY_ACCOUNTING, CO_CODE, DATE_TIME ( Các trường này đóng vai trò là KEY dữ liệu ).
INSERTnhững dữ liệu mới


Query 1:

SELECT dữ liệu từ bảng TBL_SLIP_MARK_TRANS_NON_ACC bảng group by theo INPUTTER, VALUE_DATE ( lấy theo Date ko lấy Time) , CO_CODE, BUSINESS, DEPARTMENT_CODE, COMPANY_USER ( Các trường này đóng vai trò là KEY dữ liệu ).
INSERTnhững dữ liệu mới


Query 2:

SELECT 
sm.MARK_USER mark_user,
sm.COMPANY_ACCOUNTING co_code,
AD_UNIT.UNIT_NAME unit_name,
TO_CHAR( sm.MARK_DATE, 'DD-MM-YYYY' ) mark_date,
1 number_slip
FROM
TBL_SLIP_MARK sm
JOIN AD_UNIT ON sm.COMPANY_ACCOUNTING = AD_UNIT.UNIT_CODE
WHERE
TRUNC( sm.MARK_DATE ) = TRUNC( SYSDATE - INTERVAL '1' DAY )
AND sm.MARK_USER IS NOT NULL
AND sm.SLIP_FROM = 'BI'
),
slipMarkNonAccData AS (
SELECT 
sm.MARK_USER mark_user,
sm.CO_CODE co_code,
AD_UNIT.UNIT_NAME unit_name,
TO_CHAR( sm.MARK_DATE, 'DD-MM-YYYY' ) mark_date,
1 number_slip
FROM

In [29]:
import pandas as pd

# Tạo DataFrame từ danh sách các câu lệnh SQL
df = pd.DataFrame({
    'Query Number': [f"Query {i+1}" for i in range(len(sql_codes))],
    'SQL Query': sql_codes
})

# Ghi DataFrame vào file CSV với encoding utf-8-sig
df.to_csv('sql_queries.csv', index=False, encoding='utf-8-sig')

print("SQL queries have been written to sql_queries.csv using pandas")

SQL queries have been written to sql_queries.csv using pandas


In [15]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier, Where, Comparison, Parenthesis, Function, Values
from sqlparse.tokens import Keyword, Name, Whitespace, Punctuation

def flatten_tokens(tokens):
    for token in tokens:
        if isinstance(token, (IdentifierList, Where, Parenthesis, Function)):
            for sub_token in flatten_tokens(token.tokens):
                yield sub_token
        elif token.ttype is not None or token.value.strip():
            yield token

def is_parameter(tokens, i):
    if i > 1:  # Ensure there are at least two tokens before current token
        prev_token = tokens[i - 1]
        prev_prev_token = tokens[i - 2]
        # Check if the previous two tokens are ":" and a whitespace
        if (prev_token.ttype == Whitespace and prev_token.value == ' ' and
            prev_prev_token.ttype == Punctuation and prev_prev_token.value == ':'):
            # The current token is considered a parameter
            return True
    return False

def extract_tables_and_columns(sql):
    parsed = sqlparse.parse(sql)
    tables = set()
    columns = set()
    
    # State variables to track the context of the token being processed
    from_seen = False
    join_seen = False
    where_seen = False
    group_by_seen = False
    order_by_seen = False
    insert_into_seen = False
    update_seen = False
    delete_from_seen = False
    set_seen = False

    for stmt in parsed:
        stmt_type = stmt.get_type()
        if stmt_type in {'SELECT', 'INSERT', 'UPDATE', 'DELETE'}:
            # Handle tokens for table and column extraction
            update_seen = (stmt_type == 'UPDATE')
            insert_into_seen = (stmt_type == 'INSERT')
            delete_from_seen = (stmt_type == 'DELETE')

            tokens = list(flatten_tokens(stmt.tokens))

            for i, token in enumerate(tokens):
                print(token, type(token))
                # Check and reset states based on keywords
                if token.ttype is Keyword:
                    token_value = token.value.upper()
                    if token_value == 'FROM':
                        from_seen = True
                        join_seen = False
                        where_seen = False
                        group_by_seen = False
                        order_by_seen = False
                        delete_from_seen = False
                    elif token_value == 'JOIN':
                        join_seen = True
                        from_seen = False
                    elif token_value == 'WHERE':
                        where_seen = True
                        from_seen = False
                        join_seen = False
                    elif token_value == 'GROUP BY':
                        group_by_seen = True
                        from_seen = False
                        join_seen = False
                        where_seen = False
                    elif token_value == 'ORDER BY':
                        order_by_seen = True
                        from_seen = False
                        join_seen = False
                        where_seen = False
                        group_by_seen = False
                    elif token_value == 'INSERT INTO':
                        insert_into_seen = True
                    elif token_value == 'UPDATE':
                        update_seen = True
                    elif token_value == 'DELETE FROM':
                        delete_from_seen = True
                    elif token_value == 'SET':
                        set_seen = True
                        update_seen = False

                # Collecting table names
                if update_seen or delete_from_seen:
                    if isinstance(token, Identifier):
                        tables.add(token.get_real_name())
                    elif isinstance(token, IdentifierList):
                        for identifier in token.get_identifiers():
                            tables.add(identifier.get_real_name())

                if from_seen or join_seen or insert_into_seen:
                    if isinstance(token, Identifier):
                        tables.add(token.get_real_name())
                    elif isinstance(token, IdentifierList):
                        for identifier in token.get_identifiers():
                            tables.add(identifier.get_real_name())

                # Collecting column names
                if where_seen or group_by_seen or order_by_seen or join_seen or set_seen:
                    if isinstance(token, Identifier) and not is_parameter(tokens, i):
                        columns.add(token.get_real_name())
                    elif isinstance(token, IdentifierList):
                        for identifier in token.get_identifiers():
                            columns.add(identifier.get_real_name())
                    elif isinstance(token, Comparison):
                        left, right = token.left, token.right
                        if isinstance(left, Identifier) and not is_parameter(tokens, tokens.index(left)):
                            columns.add(left.get_real_name())
                        if isinstance(right, Identifier) and not is_parameter(tokens, tokens.index(right)):
                            columns.add(right.get_real_name())

                # Collect columns in SELECT and INSERT
                if stmt_type == 'SELECT' and not any([from_seen, join_seen, where_seen, group_by_seen, order_by_seen]):
                    if isinstance(token, Identifier):
                        columns.add(token.get_real_name())
                    elif isinstance(token, IdentifierList):
                        for identifier in token.get_identifiers():
                            columns.add(identifier.get_real_name())
                elif stmt_type == 'INSERT' and insert_into_seen:
                    if isinstance(token, Identifier):
                        columns.add(token.get_real_name())
                    elif isinstance(token, IdentifierList):
                        for identifier in token.get_identifiers():
                            columns.add(identifier.get_real_name())
    
    return tables, columns

# Example usage
sql_query = """
UPDATE 
STG_SEAOPS.TBL_BI_CONTRACT_REF_DEFINE
SET
cps_tsktdk = 'Y' WHERE CUSTOMER_ID = : customerId
AND CO_CODE = : coCode
"""

tables, columns = extract_tables_and_columns(sql_query)
print("Tables:", tables)
print("Columns:", columns)



 <class 'sqlparse.sql.Token'>
UPDATE <class 'sqlparse.sql.Token'>
  <class 'sqlparse.sql.Token'>

 <class 'sqlparse.sql.Token'>
STG_SEAOPS.TBL_BI_CONTRACT_REF_DEFINE <class 'sqlparse.sql.Identifier'>

 <class 'sqlparse.sql.Token'>
SET <class 'sqlparse.sql.Token'>

 <class 'sqlparse.sql.Token'>
cps_tsktdk = 'Y' <class 'sqlparse.sql.Comparison'>


ValueError: <Identifier 'cps_ts...' at 0x26CDF7CD950> is not in list

### Tách bảng và cột từ câu lệnh SQL

In [178]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier, Where, Comparison, Parenthesis
from sqlparse.tokens import Keyword

def flatten_tokens(tokens):
    for token in tokens:
        if isinstance(token, (IdentifierList, Where, Parenthesis)):
            for sub_token in flatten_tokens(token.tokens):
                yield sub_token
        elif token.ttype is not None or token.value.strip():
            yield token

def extract_tables_and_columns(sql):
    parsed = sqlparse.parse(sql)
    tables = set()
    columns = set()
    
    # State variables to track the context of the token being processed
    from_seen = False
    join_seen = False
    where_seen = False
    group_by_seen = False
    order_by_seen = False

    for stmt in parsed:
        if stmt.get_type() == 'SELECT':
            for token in flatten_tokens(stmt.tokens):
                print(token, type(token))
                # Check and reset states based on keywords
                if token.ttype is Keyword:
                    token_value = token.value.upper()
                    print(token_value)
                    if token_value == 'FROM':
                        from_seen = True
                        join_seen = False
                        where_seen = False
                        group_by_seen = False
                        order_by_seen = False
                    elif token_value == 'JOIN':
                        join_seen = True
                        from_seen = False
                    elif token_value == 'WHERE':
                        where_seen = True
                        from_seen = False
                        join_seen = False
                    elif token_value == 'GROUP BY':
                        group_by_seen = True
                        from_seen = False
                        join_seen = False
                        where_seen = False
                    elif token_value == 'ORDER BY':
                        order_by_seen = True
                        from_seen = False
                        join_seen = False
                        where_seen = False
                        group_by_seen = False

                # Collecting table names
                if from_seen or join_seen:
                    if isinstance(token, Identifier):
                        tables.add(token.get_real_name())
                    elif isinstance(token, IdentifierList):
                        for identifier in token.get_identifiers():
                            tables.add(identifier.get_real_name())

                # Collecting column names
                if where_seen or group_by_seen or order_by_seen or join_seen:
                    if isinstance(token, Identifier):
                        columns.add(token.get_real_name())
                    elif isinstance(token, IdentifierList):
                        for identifier in token.get_identifiers():
                            columns.add(identifier.get_real_name())
                    elif isinstance(token, Comparison):
                        left, right = token.left, token.right
                        if isinstance(left, Identifier):
                            columns.add(left.get_real_name())
                        if isinstance(right, Identifier):
                            columns.add(right.get_real_name())

                # Collect columns in SELECT
                if not any([from_seen, join_seen, where_seen, group_by_seen, order_by_seen]) and isinstance(token, Identifier):
                    columns.add(token.get_real_name())
                elif not any([from_seen, join_seen, where_seen, group_by_seen, order_by_seen]) and isinstance(token, IdentifierList):
                    for identifier in token.get_identifiers():
                        columns.add(identifier.get_real_name())
    
    return tables, columns
sql_query = """
SELECT b,c
FROM a
WHERE b = 'value3';

"""
tables, columns = extract_tables_and_columns(sql_query)
print("Tables:", tables)
print("Columns:", columns)



 <class 'sqlparse.sql.Token'>
SELECT <class 'sqlparse.sql.Token'>
  <class 'sqlparse.sql.Token'>
b <class 'sqlparse.sql.Identifier'>
, <class 'sqlparse.sql.Token'>
c <class 'sqlparse.sql.Identifier'>

 <class 'sqlparse.sql.Token'>
FROM <class 'sqlparse.sql.Token'>
FROM
  <class 'sqlparse.sql.Token'>
a <class 'sqlparse.sql.Identifier'>

 <class 'sqlparse.sql.Token'>
WHERE <class 'sqlparse.sql.Token'>
WHERE
  <class 'sqlparse.sql.Token'>
b = 'value3' <class 'sqlparse.sql.Comparison'>
; <class 'sqlparse.sql.Token'>
Tables: {'a'}
Columns: {'b', 'c'}


In [81]:
# Ví dụ sử dụng
sql_query = """
SELECT LOAN.AC_AR_ID, SUM(TOT_AVL_LMT_AMT) TOT_AVL_LMT_AMT
FROM (SELECT AC_AR.AC_AR_ID
	,nvl(RI_VAL.CLT_VALT - RI_VAL.LMT_CTB, 0) TOT_AVL_LMT_AMT
FROM TWT_FNC_SVC_AR_ANL_FCT_FA_CHG AC_AR
LEFT JOIN AR_X_RI_RLTNP AR_X_RI ON AR_X_RI.AR_ID = AC_AR.AC_AR_ID
	AND AR_X_RI.EFF_DT <= AC_AR.LAST_EFF_DT
	AND AR_X_RI.END_DT > AC_AR.LAST_EFF_DT
	AND AR_X_RI.AR_X_RI_RLTNP_TP_ID = 'B6C71A2D532689A626795C8D4B67A75D' -- LOAN_AR_X_COLATERAL
	AND AR_X_RI.SRC_STM_ID = '0A173E8726B6D97003B5EE634C0CCA92' -- FCC.CLTB_ACC_COLL_LINK_DTLS
LEFT JOIN RI ON AR_X_RI.RI_ID = RI.RI_ID and RI.SRC_STM_ID = 'BB35A878C33FFBF19ADA056E5449DAAF' --FCC.GETM_COLLAT
LEFT JOIN (
	SELECT RI_VAL.RI_ID
		,NVL(MAX(DECODE(CL_CODE, 'CLT_VALT', RI_VAL.VAL_AMT)), 0) CLT_VALT
		,NVL(MAX(DECODE(CL_CODE, 'LMT_CTB', RI_VAL.VAL_AMT)), 0) LMT_CTB
	FROM RI_VAL
	JOIN CV ON RI_VAL.RI_VAL_TP_ID = CV.CL_ID
		AND CV.CL_SCM_CODE = 'RI_VAL_TP'
		AND CV.CL_CODE IN (
			'CLT_VALT'
			,'LMT_CTB'
			)
	WHERE RI_VAL.EFF_DT <= #pDate#
		AND RI_VAL.END_DT > #pDate#
	GROUP BY RI_VAL.RI_ID
	) RI_VAL ON RI_VAL.RI_ID = RI.RI_ID
LEFT JOIN RI_X_UDF_RLTNP RI_X_UDF ON RI_X_UDF.RI_ID = RI.RI_ID
	AND RI_X_UDF.EFF_DT <= #pDate#
	AND RI_X_UDF.END_DT > #pDate#
LEFT JOIN (
	SELECT RI_X_CL.*
		,CV.CL_NM
	FROM RI_X_CL_RLTNP RI_X_CL
	JOIN CV ON RI_X_CL.CL_ID = CV.CL_ID
		AND RI_X_CL.EFF_DT <= #pDate#
		AND RI_X_CL.END_DT > #pDate#
	WHERE RI_X_CL.RI_X_CL_RLTNP_TP_ID = '1AD160750012C416E42B551CC5CA4A5F' -- 'RI_X_COLL_CGY
	) COLL_CGY ON COLL_CGY.RI_ID = RI.RI_ID
LEFT JOIN AR_LC_ST LC_ST ON LC_ST.AR_ID = AR_X_RI.AR_ID
	AND LC_ST.SRC_STM_ID = '85425375A0630AA9C05A16D7D3420345' -- FCC.CLTB_ACCOUNT_APPS_MASTER
	AND LC_ST.AR_LC_ST_SCM_ID = '6B7D2ED561F31655BADF9F232E792DE2' -- TYPE: ACCOUNT_STATUS
	AND LC_ST.EFF_DT <= AC_AR.LAST_EFF_DT
	AND LC_ST.END_DT > AC_AR.LAST_EFF_DT
WHERE RI_X_UDF.UDF_ID = '890E57353540716EA2E539E6A39DBB33' -- GEDCOLLT.TCTD PHAT HANH GTCG
	AND RI_X_UDF.UDF_VAL <> 'KHAC'
	AND RI_LC_ST_TP_ID = '4C32D5E60A5794603A48F0E1AC950315' -- 'RECORD_STAT = O'
	AND LC_ST.AR_LC_ST_TP_ID IN (
		'7D108472968DF6F6A56D5BDFC01488A9'
		,'06CD0B184A47E6CEB56653555A20950F'
		) -- ACCOUNT_STATUS IN ('A', 'H')
	AND COLL_CGY.CL_NM IN (
		'121101'
		,'121102'
		,'121103'
		,'121991'
		,'121992'
		,'121993'
		,'122101'
		,'122102'
		,'122201'
		,'122202'
		,'122203'
		,'123101'
		,'123102'
		,'123201'
		,'123202'
		,'123301'
		,'123302'
		,'124001'
		,'124002'
		,'124003'
		,'125001'
		,'125002'
		,'125003'
		,'126001'
		,'126002'
		,'126003'
		,'129991'
		,'129992'
		,'129993'
		,'170301'
		,'170302'
		)) LOAN
GROUP BY LOAN.AC_AR_ID
"""

### Đưa bảng và cột vào dataframe theo alias

### Xử lý subquery

  table                  column
0  None         EVALUATION_DATE
1  None  AUTO_EVALUATION_RESULT
2  None         STATUS_SUBQUERY
3  None  TBL_CONTRACT_CONDITION
4  None         EVALUATION_DATE
5  None  AUTO_EVALUATION_RESULT
6  None           REGISTER_DATE
7  None           REGISTER_DATE
